# Vertex AI and LLamaIndex Knowledge Graph RAG Query Engine


## Graph RAG

Graph RAG is an Knowledge-enabled RAG approach to retrieve information from Knowledge Graph on given task. Typically, this is to build context based on entities' SubGraph related to the task.

## NebulaGraph Property Graph Index
NebulaGraph is an open-source distributed graph database built for super large-scale graphs with milliseconds of latency.

If you already have an existing graph, please skip to the end of this notebook.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

```python
%pip install llama-index-graph-stores-nebula
%pip install llama-index-llms-vertex
%pip install llama-index-embeddings-vertex
%pip install llama-index
```


Before we start the `Knowledge Graph RAG QueryEngine` demo, let's first get ready for basic preparation of Llama Index.

In [ ]:
# For Vertex

import os


import logging
import google.auth
import google.auth.transport.requests
from google.cloud import aiplatform
import vertexai
from vertexai.generative_models import HarmBlockThreshold, HarmCategory, SafetySetting
import sys
from dotenv import load_dotenv
# Import the Secret Manager client library.
from google.cloud import secretmanager

load_dotenv()  # this loads the .env script for use below
PROJECT_ID = os.getenv("PROJECT_ID")
LOCATION = os.getenv("LOCATION")

# try to get the secret manager stored public IP
# Create the Secret Manager client.
client = secretmanager.SecretManagerServiceClient()

# Access the secret version.
try:
    response = client.access_secret_version(request={"name": 'projects/679926387543/secrets/nebula-ip/versions/latest'})

    # Print the secret payload.
    #
    # WARNING: Do not print the secret in a production environment - this
    # snippet is showing how to access the secret material.
    payload = response.payload.data.decode("UTF-8")

    NEBULA_SERVER_ADDRESS = payload
except:
    print("no secret found, using 127.0.0.1")
    NEBULA_SERVER_ADDRESS = "127.0.0.1"

credentials = google.auth.default(quota_project_id=PROJECT_ID)[0]
request = google.auth.transport.requests.Request()
credentials.refresh(request)

safety_config = [
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=HarmBlockThreshold.BLOCK_NONE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=HarmBlockThreshold.BLOCK_NONE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=HarmBlockThreshold.BLOCK_NONE,
    ),
]

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output

vertexai.init(project=PROJECT_ID, location=LOCATION)

I0000 00:00:1730697976.702818  235254 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1730697977.516058  235254 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


## Define the LLM


In [25]:
from llama_index.llms.vertex import Vertex
from llama_index.embeddings.vertex import VertexTextEmbedding
from llama_index.core import Settings

Settings.llm = Vertex(
    temperature=0,
    model="gemini-1.5-flash",
    credentials=credentials,
    safety_settings=safety_config,
)
Settings.embed_model = VertexTextEmbedding(
    model_name="text-embedding-004", credentials=credentials
)
Settings.chunk_size = 256

# Nested Asyncio needed for async ops

In [26]:
import nest_asyncio

nest_asyncio.apply()

## Prepare for NebulaGraph

In [27]:
space_name = "gog3"

Then we could instiatate a `NebulaGraphStore`, in order to create a `StorageContext`'s `graph_store` as it.

In [28]:
%load_ext ngql
%ngql --address $NEBULA_SERVER_ADDRESS --port 9669 --user root --password <password>
%ngql CREATE SPACE IF NOT EXISTS $space_name(vid_type=FIXED_STRING(256))

The ngql extension is already loaded. To reload it, use:
  %reload_ext ngql
[OK] Connection Pool Created
INFO:nebula3.logger:Get connection to ('34.55.198.242', 9669)
INFO:nebula3.logger:Get connection to ('34.55.198.242', 9669)


""


# **One time Operation to Load the DB**

In [29]:
from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaPropertyGraphStore

graph_store = NebulaPropertyGraphStore(
    space=space_name, overwrite=True, url=f"nebula://{NEBULA_SERVER_ADDRESS}:9669"
)

Here, we assumed to have the same Knowledge Graph from [this turtorial](https://gpt-index.readthedocs.io/en/latest/examples/query_engine/knowledge_graph_query_engine.html#optional-build-the-knowledge-graph-with-llamaindex)

Let's follow on this tutorial:

# With the help of Llama Index and LLM defined, we could build Knowledge Graph from given documents.

If we have a Knowledge Graph on NebulaGraphStore already, this step could be skipped

Load data from Wikipedia for "Guardians of the Galaxy Vol. 3"

In [30]:
from llama_index.core import download_loader

from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()

documents = loader.load_data(
    pages=["Guardians of the Galaxy Vol. 3"], auto_suggest=False
)

# Next, Generate a KnowledgeGraphIndex with NebulaGraph as graph_store
Then, we will create a KnowledgeGraphIndex to enable Graph based RAG, apart from that, we have a Knowledge Graph up and running for other purposes, too!

## Create a vector store

In [31]:
from llama_index.core.vector_stores.simple import SimpleVectorStore

vec_store = SimpleVectorStore()

In [32]:
from llama_index.core.indices.property_graph import PropertyGraphIndex

index = PropertyGraphIndex.from_documents(
    documents,
    property_graph_store=graph_store,
    vector_store=vec_store,
    show_progress=True,
)

index.storage_context.vector_store.persist("./data/nebula_vec_store.json")

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 352/352 [00:01<00:00, 177.21it/s]


# Loading from an existing graph
**Important** Go here if skipping data creation step above

In [34]:
from llama_index.graph_stores.nebula import NebulaPropertyGraphStore
from llama_index.core.indices.property_graph import PropertyGraphIndex


graph_store = NebulaPropertyGraphStore(space=space_name, url=f"nebula://{NEBULA_SERVER_ADDRESS}:9669")

from llama_index.core.vector_stores.simple import SimpleVectorStore

vec_store = SimpleVectorStore.from_persist_path("./data/nebula_vec_store.json")

index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store,
    vector_store=vec_store,
)

### Now that the graph is created, we can explore it with [jupyter-nebulagraph](https://github.com/wey-gu/jupyter_nebulagraph)

In [35]:
# Query some random Relationships with Cypher

%ngql USE $space_name;
%ngql MATCH (p)-[e]->(q) RETURN e LIMIT 10

INFO:nebula3.logger:Get connection to ('34.55.198.242', 9669)
INFO:nebula3.logger:Get connection to ('34.55.198.242', 9669)


,e
0,"(""d0e04b2e-9ed2-4c2a-8719-dac82cb3cc35"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""NEXT"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""d0e04b2e-9ed2-4c2a-8719-dac82cb3cc35""}]->(""04bc8029-0024-4552-a258-4c7d4ed7d762"")"
1,"(""d0e04b2e-9ed2-4c2a-8719-dac82cb3cc35"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""PREVIOUS"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""d0e04b2e-9ed2-4c2a-8719-dac82cb3cc35""}]->(""3cfe5e88-d7e8-4492-a2f8-9a654bbce736"")"
2,"(""c966bec8-028f-4ded-8582-963fd717b2a0"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""PREVIOUS"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""c966bec8-028f-4ded-8582-963fd717b2a0""}]->(""298ad01e-3eed-4dac-be9b-1e84a06bcc92"")"
3,"(""c966bec8-028f-4ded-8582-963fd717b2a0"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""NEXT"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""c966bec8-028f-4ded-8582-963fd717b2a0""}]->(""54f17ec3-c06f-493e-bad8-c8a527708540"")"
4,"(""Vol. 3"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""Released in"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""fc5797bc-5797-4ce1-b154-5697e899197a""}]->(""2023"")"
5,"(""Vol. 3"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""Reveals"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""c966bec8-028f-4ded-8582-963fd717b2a0""}]->(""Backstory"")"
6,"(""Vol. 3"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""Fulfills"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""994caf4a-1d4a-45a6-8f2d-22f2bbda6766""}]->(""Character arc"")"
7,"(""Vol. 3"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""Is"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""e04d344a-84d8-468c-863b-46ed9744e263""}]->(""Emotional"")"
8,"(""Vol. 3"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""Put on hold"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""5196d1c1-c95b-4941-a99c-9a652ab5920d""}]->(""February 2021"")"
9,"(""Vol. 3"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""Filming"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""12768dc1-4a91-42ea-bcc0-2

In [36]:
TAGS = %ngql SHOW TAGS
%ngql SHOW TAGS

INFO:nebula3.logger:Get connection to ('34.55.198.242', 9669)
INFO:nebula3.logger:Get connection to ('34.55.198.242', 9669)


,Name
0,Chunk__
1,Entity__
2,Node__
3,Props__


In [37]:
EDGES = %ngql SHOW EDGES
%ngql SHOW EDGES

INFO:nebula3.logger:Get connection to ('34.55.198.242', 9669)
INFO:nebula3.logger:Get connection to ('34.55.198.242', 9669)


,Name
0,Relation__
1,__meta__node_label__
2,__meta__rel_label__


In [38]:
# Index the schema for querying
%ngql CREATE TAG INDEX IF NOT EXISTS entity_index on Entity__();
%ngql CREATE TAG INDEX IF NOT EXISTS props_index on Props__();
%ngql CREATE EDGE INDEX IF NOT EXISTS relation_index on Relation__();

INFO:nebula3.logger:Get connection to ('34.55.198.242', 9669)
INFO:nebula3.logger:Get connection to ('34.55.198.242', 9669)
INFO:nebula3.logger:Get connection to ('34.55.198.242', 9669)


""


In [39]:
%ngql MATCH p=(v:Entity__)-[r]->(t:Entity__) RETURN v.Entity__.name AS src, r.label AS relation, t.Entity__.name AS dest LIMIT 15;

INFO:nebula3.logger:Get connection to ('34.55.198.242', 9669)


,src,relation,dest
0,Vol. 3,Released in,2023
1,Vol. 3,Reveals,Backstory
2,Vol. 3,Fulfills,Character arc
3,Vol. 3,Is,Emotional
4,Vol. 3,Put on hold,February 2021
5,Vol. 3,Filming,February to late april 2022
6,Vol. 3,Is,First mcu film
7,Vol. 3,Uses,Flashbacks
8,Vol. 3,Tonal difference,Holiday special
9,Vol. 3,Filming,Late 2021


In [128]:
%ng_draw

<class 'pyvis.network.Network'> |N|=16 |E|=15

### The rendered output should look like this:

![](./graph_output.png)

# Querying and Retrieval
1. Getting a simple graph from a query

In [40]:
subgraph_retriever = index.as_retriever(
    include_text=False,  # include source text in returned nodes, default True
)

In [41]:
from IPython.display import display, Markdown

nodes = subgraph_retriever.retrieve(
    "Tell me about the production of Guardians of the Galaxy 3",
)
node_text = ""
for node in nodes:
    display(Markdown(f"{node.text}"))

Guardians of the galaxy vol. 3 -> Dethroned -> Fast x

Guardians of the galaxy vol. 3 -> Directed -> Gunn

Guardians of the galaxy vol. 3 -> Projected to gross -> $110 million

Guardians of the galaxy vol. 3 -> Debuted with -> $118.4 million

Guardians of the galaxy vol. 3 -> Net profit -> $124 million

...

The making of guardians of the galaxy vol. 3 -> Is -> Special

Gunn chose practical effects for vol. 3 -> Said -> Beth mickle

Story of this iteration of the guardians of the galaxy -> Conclude -> Film

Return to write and direct guardians of the galaxy vol. 3 -> Announced -> Gunn

Return to write and direct guardians of the galaxy vol. 3 -> Announced -> James gunn

2. Getting a NL query over the graph

In [42]:
query_engine = index.as_query_engine(
    streaming=True,
    include_text=True,  # include source text in returned nodes, default True
)

In [43]:
response = query_engine.query(
    "Tell me about the production of Guardians of the Galaxy 3 and it's relation to James Gunn?",
)
display(Markdown(response.print_response_stream()))

James Gunn announced in 2017 that he would return to write and direct Guardians of the Galaxy Vol. 3. He said the film would be set after Avengers: Infinity War and Avengers: Endgame, and would "conclude the story of this iteration of the Guardians of the Galaxy, and help catapult both old and new Marvel characters into the next ten years and beyond". He also felt that the three Guardians films would "work together as a whole", telling one story, with the third film "tying a lot of stuff together" from the first two and giving "a lot of answers on a lot of different things". Gunn also planned to work with Marvel on the future of the "Marvel Cosmic Universe". He was set to begin work on Vol. 3, but was fired by Disney in July 2018 after old tweets of his resurfaced. Despite this, Gunn was rehired by Disney in October 2019 and returned to direct Guardians of the Galaxy Vol. 3. 


<IPython.core.display.Markdown object>

#### Check on source nodes

In [44]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='4ea2b508-ccc9-40b8-8d50-62f1400ea221', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='51126744', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='fd9ffa2f4b966406369dcea5703227e5ff64dcf6534cb232c1b41552186b3b03'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='aff90cc3-d378-4b5e-a3ec-8babd46063e6', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='b72ffe073abc4ac3e8bc94e1cdbe96e2d43fc85e2d52946a0c606a0a073defb6'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='64dc6745-d313-412d-b95d-416d6da3acca', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='13704853a2486d9fbd3c4d9844372c71bbfc359e729e83328bb4ac03baba7f39')}, text='Here are some facts extracted from the provided text:\n\nGuardians of the galaxy vol. 3 -> Dethroned -> Fast x\nGuardians of the galaxy vol. 3 -> Directed -> Gunn\nGuardians of the gala

In [45]:
response.get_formatted_sources()

'> Source (Node id: 4ea2b508-ccc9-40b8-8d50-62f1400ea221): Here are some facts extracted from the provided text:\n\nGuardians of the galaxy vol. 3 -> Dethrone...\n\n> Source (Node id: 6612e694-2e48-405f-bd2d-4809c8ca3f26): Here are some facts extracted from the provided text:\n\nThe making of guardians of the galaxy vol....\n\n> Source (Node id: 4f250b76-5236-427d-bc55-b2e30827f314): Here are some facts extracted from the provided text:\n\nGunn chose practical effects for vol. 3 ->...\n\n> Source (Node id: ab13c0c6-c364-4a59-a2d2-94b66748b7ce): Here are some facts extracted from the provided text:\n\nReturn to write and direct guardians of th...\n\n> Source (Node id: 25994849-f3a6-46fb-ab3c-fccaf2c69753): Here are some facts extracted from the provided text:\n\nRumors gunn replaced established marvel di...'

3. Using a hybrid query engine

In [46]:
query_engine = index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
)

In [47]:
response = query_engine.query(
    "Tell me about the production of Guardians of the Galaxy 3?",
)
display(Markdown(f"<b>{response}</b>"))

<b>The film was directed by James Gunn and produced by Marvel Studios. It was released on May 5th and is a sequel to Guardians of the Galaxy and Guardians of the Galaxy Vol. 2. The film features Chris Pratt, Dave Bautista, Karen Gillan, and Bradley Cooper. It is set after Avengers: Infinity War and Avengers: Endgame. 
</b>

In [48]:
response = await query_engine.aquery(
    "Tell me about Starlord's relationship to Groot?",
)
display(Markdown(f"<b>{response}</b>"))

<b>Star-Lord and Groot are members of the Guardians of the Galaxy and are close friends. 
</b>

4. Use `TextToCypherRetriever` to generate queries

In [49]:
from llama_index.core.indices.property_graph import TextToCypherRetriever


DEFAULT_RESPONSE_TEMPLATE = (
    "Generated Cypher query:\n{query}\n\n" "Cypher Response:\n{response}"
)
DEFAULT_ALLOWED_FIELDS = ["text", "label", "type"]

DEFAULT_TEXT_TO_CYPHER_TEMPLATE = (index.property_graph_store.text_to_cypher_template,)


cypher_retriever = TextToCypherRetriever(
    index.property_graph_store,
    # customize the LLM, defaults to Settings.llm
    llm=Settings.llm,
    # customize the text-to-cypher template.
    # Requires `schema` and `question` template args
    text_to_cypher_template=index.property_graph_store.text_to_cypher_template,
    # customize how the cypher result is inserted into
    # a text node. Requires `query` and `response` template args
    response_template=DEFAULT_RESPONSE_TEMPLATE,
    # an optional callable that can clean/verify generated cypher
    cypher_validator=None,
    # allowed fields in the resulting
    allowed_output_field=DEFAULT_ALLOWED_FIELDS,
)

In [50]:
SCHEMA = {
    "nodes": TAGS,
    "edges": EDGES,
}

In [51]:
nodes = cypher_retriever.retrieve(
    index.property_graph_store.text_to_cypher_template.format(
        question="Tell me about the production of Gaurdians of the Galaxy 3",
        schema=str(SCHEMA),
    )
)

In [52]:
for node in nodes:
    print(node)

Node ID: 753c3a00-4b9d-49cc-842f-3644dceeb50b
Text: Generated Cypher query: MATCH (m:Entity__ {name: "Guardians of
the Galaxy 3"})<-[:Relation__]-(p:Props__) RETURN p   Cypher Response:
[]
Score:  1.000



# Cleanup 
%ngql
CLEAR/DROP SPACE $space_name; 